In [0]:
dbutils.widgets.text("env", "dev", "Environment")

env = dbutils.widgets.get("env")

In [0]:
%run ../config $env=$env

In [0]:
from pyspark.sql import functions as F

In [0]:
df_drivers = (
    spark.sql("SELECT * FROM bronze_tbl_drivers")
    .select("driver_id", F.col("code").alias("driver_code"))
)

display(df_drivers.limit(2)) if env == "dev" else None

In [0]:
df_constructors = (
    spark.sql("SELECT * FROM bronze_tbl_constructors")
    .select("constructor_id", F.col("name").alias("constructor_name"))
)

display(df_constructors.limit(2)) if env == "dev" else None

In [0]:
df_races = (
    spark.sql("SELECT * FROM bronze_tbl_races")
    .select("race_id", "year")
)

display(df_races.limit(2)) if env == "dev" else None

In [0]:
df_results = (
    spark.sql("SELECT * FROM bronze_tbl_results")
    .select("race_id", "constructor_id", "driver_id", "points")
)

display(df_results.limit(2)) if env == "dev" else None

In [0]:
df = (
    df_races
    .join(df_results, on="race_id", how="inner")
    .join(df_drivers, on="driver_id", how="inner")
    .join(df_constructors, on="constructor_id", how="inner")
    .drop("driver_id", "constructor_id")
    .withColumnRenamed("driver_code", "driver")
    .withColumnRenamed("constructor_name", "constructor")
)

display(df.limit(2)) if env == "dev" else None

In [0]:
df_driver_standings = (
    df
    .select("year", "driver", "points")
    .groupBy("year", "driver")
    .agg(F.sum("points").alias("points"))
    .orderBy(F.desc("year"), F.desc("points"))
)

df_driver_standings.display()

In [0]:
df_constructor_standings = (
    df
    .select("year", "constructor", "points")
    .groupBy("year", "constructor")
    .agg(F.sum("points").alias("points"))
    .orderBy(F.desc("year"), F.desc("points"))
)

df_constructor_standings.display()

In [0]:
#df_driver_standings.write.format("delta").saveAsTable("gold_tbl_driver_standings", mode="overwrite")